# Deep learning applications in medical imaging: Ex.2

Due date: 9/5/23

You can submit as one student or in pairs (two students).

Submitted by: [ **Rafael Ashurov 312054711, Corinne Golan 302276159** ]


# 1. Buidling a simple convolution neural network.

Your task is to build a classifier to classify X-ray images into 4 categories: Normal, Cvoid-19 infection, Lung opacity, Viral pneumonia.

If you want to look at the X-ray images in the data set you can download them from here:
http://dl.dropboxusercontent.com/s/6hq5t0sbbx6n1g2/medical_xray.zip?dl=0
Or you can look at the environment files in Google Colab after running the second cell (double click on an image to view it).

*Notice that unfortantly we don't have many samples but try to do your best and may the force be with you.

- If you are using Google Colab switch to GPU mode by selecting "Runtime" on the top menu and then "Change runtime type", in "Hardware accelerator" select "GPU".

- If you are using Google Colab and suddenly encounter unexpected problems (even sudden drop in accuracy) try selecting in the top menu "Runtime"->"Disconnect and delete runtime", after that run all the cells again.

- First run the code cells below which do the following: contain the necessary imports, retrieve the data, process it into train and test data sets. Look at the comments and verify you get the prints right to verify all works fine.

In [1]:
import pathlib
import os
import matplotlib.pyplot as plt
import numpy as np
import requests, zipfile, io
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, Input
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from PIL import Image, ImageOps

In [2]:
#Download dataset from URL as zip and extract its content to Google Colab environment
r = requests.get('http://dl.dropboxusercontent.com/s/6hq5t0sbbx6n1g2/medical_xray.zip?dl=0', stream=True)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [3]:
#Iterate the sub folder and print the classes - *total 4 classes*
# You should see the following print:
# ['covid', 'lung_opacity', 'normal', 'viral_pneumonia']

PATH = '/content/medical_xray'

p = pathlib.Path(PATH)

labels = [x.parts[-1] for x in p.iterdir()]

num_classes = len(labels)
labels.sort()
print(labels)

['covid', 'lung_opacity', 'normal', 'viral_pneumonia']


In [4]:
# Print number of samples for each class - *about 224 samples*
# You should see the following prints:
# covid 224
# lung_opacity 224
# normal 224
# viral_pneumonia 224

files_path = []
files_labels = []

for root, dirs, files in os.walk(PATH):
  p = pathlib.Path(root)
  
  for file in files:
    files_path.append(root + '/' + file)
    files_labels.append(p.parts[-1])

label_cnt = []

for label in labels:
  print(label, files_labels.count(label))
  label_cnt.append(files_labels.count(label))

covid 224
lung_opacity 224
normal 224
viral_pneumonia 224


In [5]:
# Process the data, split it to 80% of the samples for train, 20% test.

file_path = []
file_label = []
for root,dirs,files in os.walk(PATH):
    p = pathlib.Path(root)
    for file in files:
        file_path.append(root+"/"+file)
        file_label.append(p.parts[-1])

X = []
y = []
for path in file_path:
    img = Image.open(path)
    img = ImageOps.grayscale(img)
    if (img.width or img.height) != 64:
        img = img.resize((64,64))
    img_arr = np.asarray(img,dtype  = np.int16)
    X.append(img_arr)
X = np.array(X)
y = np.array(file_label)

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 1)

y_train = list(map(lambda x : labels.index(x),y_train))
y_train = np.array(y_train)

y_test = list(map(lambda x: labels.index(x),y_test))
y_test = np.array(y_test)

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

#Print train shape and number of train and test samples, you should see the following prints:
# x_train shape: (716, 64, 64, 1)
# 716 train samples
# 180 test sample
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

x_train shape: (716, 64, 64, 1)
716 train samples
180 test samples


- Build a simple convolution neural network (like we saw on class) with a maximum of 5 layers or less (Only Conv2D and Dense counts as layers).

- You can **only use** the following components: Conv2D, MaxPooling2D, Flatten, Dense.

- Use sgd_momentum with learning_rate=0.01, momentum=0.9, use loss='sparse_categorical_crossentropy'

- Run for 50 epochs, take approxmatly the 5 best epochs in terms of accuracy and write their average as the result, you should get a result of at least 80% accuracy.

- Take notice about the input shape, number of classes and batch size in your code. Also notice that accuracy is not the only measure of success  here - meaning you can choose wrong values and stil get a good accuracy (in this case I will not give a you full score).

In [6]:
def print_avrg_val_acc (history):
    # Find the 5 best epochs in terms of validation accuracy
    val_acc = history.history['val_accuracy']
    best_epochs = sorted(val_acc, reverse=True)[:5]
    avg_val_acc = sum(best_epochs)/5
    print("Average validation accuracy of the 5 best epochs: ", avg_val_acc)

In [13]:
from tensorflow.keras.optimizers import SGD

# Define the model
model1 = Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(64,64,1)),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(units=256, activation='relu'),
    Dense(units=4, activation='softmax')
])
print(model1.summary())

# Compile the model using gradient descent WITH MOMENTUM:
sgd_momentum = SGD(learning_rate=0.01, momentum=0.9)
model1.compile(optimizer=sgd_momentum, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model1.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_test, y_test))

print_avrg_val_acc(history)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 31, 31, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 14, 14, 64)       0         
 g2D)                                                            
                                                                 
 flatten_5 (Flatten)         (None, 12544)             0         
                                                                 
 dense_10 (Dense)            (None, 256)              

- Try to improve your result by at least 2% with any of the methods we saw in class, beside augmentation and beside changing the number of layers or number/sizes of the kernels (since those methods are found in the next questions).

In [16]:
from keras.layers import BatchNormalization

# Define the model
model2 = Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(64,64,1)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(units=256, activation='relu'),
    Dense(units=4, activation='softmax')
])
print(model2.summary())
sgd_momentum = SGD(learning_rate=0.005, momentum=0.9)
model2.compile(optimizer=sgd_momentum, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history2 = model2.fit(x_train, y_train, epochs=50, batch_size=16, validation_data=(x_test, y_test))

print_avrg_val_acc(history2)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 62, 62, 32)        320       
                                                                 
 batch_normalization_14 (Bat  (None, 62, 62, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 31, 31, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 29, 29, 64)        18496     
                                                                 
 batch_normalization_15 (Bat  (None, 29, 29, 64)       256       
 chNormalization)                                                
                                                      

- Try changing the number of layers in the network, also try changing the number of kernels or their sizes (take inspiration from the more complex modles we saw on class)

- Does changing the number of layers and changing the kernel sizes/numbers improved the result or not? explain why?

<font color="#FFD629">
<b><u> Increase pooling kernel size: </b></u>
</font>

In [9]:
# Define the model
model3 = Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(64,64,1)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(5,5)),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(5,5)),
    Flatten(),
    Dense(units=256, activation='relu'),
    Dense(units=4, activation='softmax')
])
print(model3.summary())
sgd_momentum = SGD(learning_rate=0.005, momentum=0.9)
model3.compile(optimizer=sgd_momentum, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history3 = model3.fit(x_train, y_train, epochs=50, batch_size=16, validation_data=(x_test, y_test))

print_avrg_val_acc(history3)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 62, 62, 32)        320       
                                                                 
 batch_normalization_2 (Batc  (None, 62, 62, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 10, 10, 64)        18496     
                                                                 
 batch_normalization_3 (Batc  (None, 10, 10, 64)       256       
 hNormalization)                                                 
                                                      

<font color="#5fbefe">
When we increased the MaxPooling kernel size we got better results. The kernel size determines the size of the region over which the maximum value is calculated, so when we set hte kernel bigger the window calculates the maximum value over larger regions of the input image, which helps to retain more information and detail. This can lead to better performance in some cases, but it may also result in a larger output size and slower computation
</font>

<font color="#FFD629">
Increase number of layers:
</font>

In [17]:
# Define the model
model4 = Sequential([
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=(64,64,1)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(filters=256, kernel_size=(3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(filters=512, kernel_size=(3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(units=1024, activation='relu'),
    Dense(units=4, activation='softmax')
])
print(model4.summary())
sgd_momentum = SGD(learning_rate=0.01, momentum=0.9)
model4.compile(optimizer=sgd_momentum, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history4 = model4.fit(x_train, y_train, epochs=50, batch_size=16, validation_data=(x_test, y_test))

print_avrg_val_acc(history4)

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 62, 62, 64)        640       
                                                                 
 batch_normalization_16 (Bat  (None, 62, 62, 64)       256       
 chNormalization)                                                
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 31, 31, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 29, 29, 128)       73856     
                                                                 
 batch_normalization_17 (Bat  (None, 29, 29, 128)      512       
 chNormalization)                                                
                                                      

<font color="#5fbefe">
Here we got better results than the simple 5 layer model with the 3x3 pool kernel, but worse than the results of the 5x5 pool kernel. 
The result is better than the simple model because it uses more layers and can detect more variance in our images correctly, but, because of the higher number of layers the model became more complex and starts to fit the training data more than the simple model with the 5x5 pooling kernel.
</font>

# 2. Using augmentation

- Try using augmentation, even if its not helpful and you get worst results don't worry about it, we are just experimenting.

- Read the description in this link: https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
(in the section titled "Data pre-processing and data augmentation") to understand better how to use augmentation in Keras (along side what we studied at class).

- Choose reasonable values for the augmentation considring the data we  are working on, for example rotation_range = 180 is not reasonable since we are not expected to recieve a flipped X-ray image.

- If you want to look at the X-ray images in the data you can download them from here:
http://dl.dropboxusercontent.com/s/6hq5t0sbbx6n1g2/medical_xray.zip?dl=0
Or you can look at the environment files in Google Colab after running the second cell (double click on an image to view it).

In [11]:
# We will use a model with batch normalization and 5x5 pooling kernel because 
# it had the best results
model = Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(64,64,1)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(5,5)),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(5,5)),
    Flatten(),
    Dense(units=256, activation='relu'),
    Dense(units=4, activation='softmax')
])

sgd_momentum = SGD(learning_rate=0.005, momentum=0.9)
model.compile(optimizer=sgd_momentum, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,
    vertical_flip=False
    )

data_iter = datagen.flow(x_train, y_train, batch_size=32)

history = model.fit(data_iter, steps_per_epoch=x_train.shape[0]//32, epochs=50, 
                    validation_data=(x_test, y_test), verbose=0)

print_avrg_val_acc(history)

Average validation accuracy of the 5 best epochs:  0.8144444346427917


## Good Luck!